In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astroML import stats as astroMLstats
from sklearn.neighbors import KernelDensity
import scipy.stats as stats
import scipy.integrate
import seaborn as sns
from tqdm import tqdm

ModuleNotFoundError: No module named 'astroML'